In [2]:
import pandas as pd

# Load CSV into DataFrame
parent_folder = 'NYPL-menus 2'
dish = pd.read_csv(f"{parent_folder}/Dish.csv")
menu = pd.read_csv(f"{parent_folder}/Menu.csv")
menu_item = pd.read_csv(f"{parent_folder}/MenuItem.csv")
menu_page = pd.read_csv(f"{parent_folder}/MenuPage.csv")

display(dish)
# display(menu)
# display(menu_item)
# display(menu_page)

,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
0,1,Consomme printaniere royal,NaN,8,8,1897,1927,0.20,0.4
1,2,Chicken gumbo,NaN,111,117,1895,1960,0.10,0.8
2,3,Tomato aux croutons,NaN,13,13,1893,1917,0.25,0.4
3,4,Onion au gratin,NaN,41,41,1900,1971,0.25,1.0
4,5,St. Emilion,NaN,66,68,1881,1981,0.00,18.0
...,...,...,...,...,...,...,...,...,...
423392,515673,Boiled: Corned beef & cabbage,NaN,1,1,0,0,0.00,0.0
423393,515674,Boiled: Knuckle of Veal & Bacon,NaN,1,1,0,0,0.00,0.0
423394,515675,Roast: Turkey & Cranberry Sauce,NaN,1,1,0,0,0.00,0.0
423395,515676,"Claret: Chateau Larose, Cruse et Fils Freres",NaN,1,1,0,0,0.00,0.0


# Data Profiling

In [12]:
dish.describe()

,id,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
count,423397.000000,0.0,423397.000000,423397.000000,423397.000000,423397.000000,394297.000000,394297.000000
mean,264456.594900,NaN,3.060489,3.146794,1675.514555,1679.299738,0.965265,1.603875
std,150489.070889,NaN,27.818178,29.962122,651.321461,651.934580,6.714564,12.696274
min,1.000000,NaN,0.000000,-6.000000,0.000000,0.000000,0.000000,0.000000
25%,132374.000000,NaN,1.000000,1.000000,1900.000000,1900.000000,0.000000,0.000000
50%,269636.000000,NaN,1.000000,1.000000,1914.000000,1917.000000,0.000000,0.000000
75%,397135.000000,NaN,1.000000,1.000000,1949.000000,1955.000000,0.400000,0.600000
max,515677.000000,NaN,7740.000000,8484.000000,2928.000000,2928.000000,1035.000000,3050.000000


In [6]:
df = dish
df.info(verbose=True)

columns_with_mv = df.columns[df.isnull().any()]
df[columns_with_mv].isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423397 entries, 0 to 423396
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              423397 non-null  int64  
 1   name            423397 non-null  object 
 2   description     0 non-null       float64
 3   menus_appeared  423397 non-null  int64  
 4   times_appeared  423397 non-null  int64  
 5   first_appeared  423397 non-null  int64  
 6   last_appeared   423397 non-null  int64  
 7   lowest_price    394297 non-null  float64
 8   highest_price   394297 non-null  float64
dtypes: float64(3), int64(5), object(1)
memory usage: 29.1+ MB


description      423397
lowest_price      29100
highest_price     29100
dtype: int64

# Cleaning Verification

In [17]:
# null count as percentage or sum
fields = ['description']
null = dish[fields].isnull()
counts = null.sum()

print("=== Count ===")
print(counts)
print("=== Percent ===")
print(null.mean())

=== Count ===
description    423397
dtype: int64
=== Percent ===
description    1.0
dtype: float64


In [14]:
# IC Violations and Constraints

# check for duplicate rows
df = dish
dups = df[df.duplicated(subset=['id'], keep=False)]
display(dups)

# FK and PK
counts = df.groupby('id')['name'].nunique()
bad_id = counts[counts > 1].index
bad_df = df[df['id'].isin(bad_id)]
display(bad_df)

ic3_violations = bad_df.groupby(['id', 'name']).size().to_dict()
ic3_violations

,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price


,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price


{}

# Data Cleaning

In [ ]:
df = dish
df['name_trim'] = df['name'].str.strip()
df['host_name_trim'] = df['host_name'].str.strip()
df['name_upper'] = df['name_trim'].str.upper()
df_wt_outliers = df[df['minimum_nights'] < 400]
df['new_last_review'] = pd.to_datetime(df['last_review'], format='%m/%d/%y', errors='coerce')
avg = df['number_of_reviews'].astype(float).mean()
df['no_reviews_wt_missing'] = df['number_of_reviews'].fillna(avg).astype(float)